## 1. 多模态大模型识别 PDF 中的图片

[通义千问 VL-Max 模型](https://bailian.console.aliyun.com/?accounttraceid=ca4a146a1a7042369aa49e4377539ef9bgjd#/model-market/detail/qwen-vl-max-1230)


In [8]:
# pip install -U openai
import os
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="qwen-vl-ocr",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": "https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241108/ctdzex/biaozhun.jpg",
                    "min_pixels": 28 * 28 * 4,
                    "max_pixels": 1280 * 784
                },
                # 目前为保证识别效果，模型内部会统一使用"Read all the text in the image."作为text的值，用户输入的文本不会生效。
                {"type": "text", "text": "Read all the text in the image."},
            ]
        }
    ],
    top_p=0.01,
    temperature=0.1,
    max_tokens=2000)

# 转为 json 格式继续处理
print(completion.model_dump_json())

{"id":"chatcmpl-46c1a1f3-4f71-9f2d-9ca7-e7157100c19f","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"读者对象 如果你是Linux环境下的系统管理员，那么学会编写shell脚本将让你受益匪浅。本书并未细述安装 Linux系统的每个步骤，但只要系统已安装好Linux并能运行起来，你就可以开始考虑如何让一些日常 的系统管理任务实现自动化。这时shell脚本编程就能发挥作用了，这也正是本书的作用所在。本书将 演示如何使用shell脚本来自动处理系统管理任务，包括从监测系统统计数据和数据文件到为你的老板 生成报表。 如果你是家用Linux爱好者，同样能从本书中获益。现今，用户很容易在诸多部件堆积而成的图形环境 中迷失。大多数桌面Linux发行版都尽量向一般用户隐藏系统的内部细节。但有时你确实需要知道内部 发生了什么。本书将告诉你如何启动Linux命令行以及接下来要做什么。通常，如果是执行一些简单任 务(比如文件管理) ， 在命令行下操作要比在华丽的图形界面下方便得多。在命令行下有大量的命令 可供使用，本书将会展示如何使用它们。","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1736354873,"model":"qwen-vl-ocr","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":225,"prompt_tokens":1248,"total_tokens":1473,"completion_tokens_details":null,"prompt_tokens_details":null}}


In [9]:
print(completion.choices[0].message.content)

读者对象 如果你是Linux环境下的系统管理员，那么学会编写shell脚本将让你受益匪浅。本书并未细述安装 Linux系统的每个步骤，但只要系统已安装好Linux并能运行起来，你就可以开始考虑如何让一些日常 的系统管理任务实现自动化。这时shell脚本编程就能发挥作用了，这也正是本书的作用所在。本书将 演示如何使用shell脚本来自动处理系统管理任务，包括从监测系统统计数据和数据文件到为你的老板 生成报表。 如果你是家用Linux爱好者，同样能从本书中获益。现今，用户很容易在诸多部件堆积而成的图形环境 中迷失。大多数桌面Linux发行版都尽量向一般用户隐藏系统的内部细节。但有时你确实需要知道内部 发生了什么。本书将告诉你如何启动Linux命令行以及接下来要做什么。通常，如果是执行一些简单任 务(比如文件管理) ， 在命令行下操作要比在华丽的图形界面下方便得多。在命令行下有大量的命令 可供使用，本书将会展示如何使用它们。


## 2. 成熟的工业级 RAG 实现
### 2.1 QAnything 的OCR 识别、两段式 Rerank

[QAnything 官方文档](https://github.com/netease-youdao/QAnything/blob/qanything-python/README_zh.md) 


#### 1 QAnything 的软件：

- QAnything 使用 sanic 实现 web 服务 ，Local_RAG 使用 FastAPI 实现，FastAPI 更广泛
- 向量检索模型 bce-embedding-base_v1 + 向量排序模型 bce-reranker-base_v1 组合
- 业务数据库使用 MySQL，向量数据库使用 faiss

#### 2 从哪里开始看源代码：

从 API 作为入口，跟着 RAG 原理，从本地文档检索开始，到向量检索，再到向量排序，最后到 RAG 结果



In [ ]:
# 1 文件上传
# QAnything-python\qanything_kernel\qanything_server\sanic_api.py
# app.add_route(upload_files, "/api/local_doc_qa/upload_files", methods=['POST'])  # tags=["上传文件"] 
# 鼠标右键，转到定义

# 2 pdf 处理
# QAnything-python\qanything_kernel\core\local_file.py
# elif self.file_path.lower().endswith(".pdf"):

# 3 加载PDF,如果需要 OCR 执行新的路线
# QAnything-python\qanything_kernel\utils\loader\pdf_loader.py
# def pdf_ocr_txt(filepath, dir_path="tmp_files"):

# 4 清理特殊字符
# QAnything-python\qanything_kernel\core\local_file.py
# texts_splitter = ChineseTextSplitter(pdf=True, sentence_size=sentence_size)

# 5 文件检索
# QAnything-python\qanything_kernel\core\local_doc_qa.py
# async def get_source_documents(self, query, kb_ids, cosine_thresh=None, top_k=None):

# 6 向量数据库搜索
# QAnything-python\qanything_kernel\connector\database\faiss\faiss_client.py
# docs_with_score = await self.faiss_client.asimilarity_search_with_score(query, k=top_k, filter=filter,
#                                                                                fetch_k=200)

## 7 检索结果重排
# QAnything-python\qanything_kernel\core\local_doc_qa.py
# def rerank_documents(self, query, source_documents):


### 2.2 Dify 多路召回机制与 Workflow

[召回模式](https://github.com/langgenius/dify-docs/blob/main/zh_CN/learn-more/extended-reading/retrieval-augment/retrieval.md)

在多路召回模式下，检索器会在所有与应用关联的知识库中去检索与用户问题相关的文本内容，并将多路召回的相关文档结果合并，并通过 Rerank 模型对检索召回的文档进行语义重排序。

由于多路召回模式不依赖于模型的推理能力或知识库描述，该模式在多知识库检索时能够获得质量更高的召回效果，除此之外加入 Rerank 步骤也能有效改进文档召回效果。因此，当创建的知识库问答应用关联了多个知识库时，我们更推荐将召回模式配置为多路召回。


[工作流](https://docs.dify.ai/zh-hans/guides/workflow)

Dify 工作流分为两种类型：

Chatflow：面向对话类情景，包括客户服务、语义搜索、以及其他需要在构建响应时进行多步逻辑的对话式应用程序。

Workflow：面向自动化和批处理情景，适合高质量翻译、数据分析、内容生成、电子邮件自动化等应用程序。


### 2.3 如何通过变量和知识库与 Dify 结合